ETL DEL SISTEMA DE RECOMENDACION DE PELICULAS: LIMPIEZA, DESANIDADO, NORMALIZACION Y LIMPIEZA DE DATOS

Importamos las librerias a usar

In [1]:
import pandas as pd
import numpy as np
from pandas import json_normalize
import ast
from datetime import datetime

Importamos los archivos csv y los pasamos a dataframe 

In [2]:
read_movies = r'C:\Users\EFRAIN LINARES\Desktop\Proyecto 1\movies_dataset.csv'
read_credits = r'C:\Users\EFRAIN LINARES\Desktop\Proyecto 1\credits.csv'

In [3]:
df_movies = pd.read_csv(read_movies)
df_credits = pd.read_csv(read_credits)

C:\Users\EFRAIN LINARES\AppData\Local\Temp\ipykernel_18288\2537683332.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv(read_movies)


Examinamos los datos para ver si hay valores nulos e ir viendo que columnas vamos a tener que eliminar

In [4]:
df_movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [6]:
df_credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [7]:
df_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


LIMPIEZA DE DATOS

Procedemos a eliminar las columnas "adult","video","imdb_id","original_title","poster_path" y "homepage", ya que estos nos lo es un requisito para poder avanzar en el proyecto y al igual que la columna "tagline" ya que son datos que no vamos a necesitar

In [8]:
column_to_drop = ['adult','video','imdb_id','original_title','poster_path','homepage','tagline']
df_movies = df_movies.drop(columns=column_to_drop)
df_movies

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Toy Story,7.7,5415.0
1,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Jumanji,6.9,2413.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Grumpier Old Men,6.5,92.0
3,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Waiting to Exhale,6.1,34.0
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Father of the Bride Part II,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",439050,fa,Rising and falling between a man and woman.,0.072051,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Subdue,4.0,1.0
45462,NaN,0,"[{'id': 18, 'name': 'Drama'}]",111109,tl,An artist struggles to finish his work while a...,0.178241,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,Century of Birthing,9.0,3.0
45463,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Betrayal,3.8,6.0
45464,NaN,0,[],227506,en,"In a small town live two brothers, one a minis...",0.003503,"[{'name': 'Yermoliev', 'id': 88753}]","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",1917-10-21,0.0,87.0,[],Released,Satan Triumphant,0.0,0.0


<b>LIMPIEZA DE LOS DATOS</b>

Eliminamos valores duplicados

In [9]:
df_movies = df_movies.drop_duplicates()
df_credits = df_credits.drop_duplicates()

In [10]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45453 entries, 0 to 45465
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4494 non-null   object 
 1   budget                 45453 non-null  object 
 2   genres                 45453 non-null  object 
 3   id                     45453 non-null  object 
 4   original_language      45442 non-null  object 
 5   overview               44499 non-null  object 
 6   popularity             45448 non-null  object 
 7   production_companies   45450 non-null  object 
 8   production_countries   45450 non-null  object 
 9   release_date           45366 non-null  object 
 10  revenue                45447 non-null  float64
 11  runtime                45190 non-null  float64
 12  spoken_languages       45447 non-null  object 
 13  status                 45366 non-null  object 
 14  title                  45447 non-null  object 
 15  vote_av

In [11]:
#Los valores nulos del campo release date deben eliminarse.
df_movies.dropna(subset=['release_date'], inplace=True)

#Los datos que estén ausentes en las columnas serán eliminados si no representan más del 3% del total de registros. 
#Esto se realizará para evitar la eliminación de filas en su totalidad utilizando el método .dropna(), lo cual podría afectar significativamente el dataset.

df_movies.dropna(subset=['overview'], inplace=True)
df_movies.dropna(subset=['popularity'], inplace=True)
df_movies.dropna(subset=['production_companies'], inplace=True)
df_movies.dropna(subset=['production_countries'], inplace=True)
df_movies.dropna(subset=['revenue'], inplace=True)
df_movies.dropna(subset=['runtime'], inplace=True)
df_movies.dropna(subset=['spoken_languages'], inplace=True)
df_movies.dropna(subset=['status'], inplace=True)
df_movies.dropna(subset=['title'], inplace=True)
df_movies.dropna(subset=['vote_average'], inplace=True)
df_movies.dropna(subset=['vote_count'], inplace=True)

C:\Users\EFRAIN LINARES\AppData\Local\Temp\ipykernel_18288\956912735.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies.dropna(subset=['release_date'], inplace=True)
C:\Users\EFRAIN LINARES\AppData\Local\Temp\ipykernel_18288\956912735.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies.dropna(subset=['overview'], inplace=True)
C:\Users\EFRAIN LINARES\AppData\Local\Temp\ipykernel_18288\956912735.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

In [12]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44358 entries, 0 to 45465
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4428 non-null   object 
 1   budget                 44358 non-null  object 
 2   genres                 44358 non-null  object 
 3   id                     44358 non-null  object 
 4   original_language      44348 non-null  object 
 5   overview               44358 non-null  object 
 6   popularity             44358 non-null  object 
 7   production_companies   44358 non-null  object 
 8   production_countries   44358 non-null  object 
 9   release_date           44358 non-null  object 
 10  revenue                44358 non-null  float64
 11  runtime                44358 non-null  float64
 12  spoken_languages       44358 non-null  object 
 13  status                 44358 non-null  object 
 14  title                  44358 non-null  object 
 15  vote_av

<b>TRANSFORMACIÓN DE DATOS</b>

In [13]:
# Los valores nulos de las columnas "budget" y "revenue" tienen que ser reemplazados por valores 0
df_movies["budget"] = df_movies["budget"].fillna(0)
df_movies["revenue"] = df_movies['revenue'].fillna(0)

df_movies

C:\Users\EFRAIN LINARES\AppData\Local\Temp\ipykernel_18288\2352164090.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies["budget"] = df_movies["budget"].fillna(0)
C:\Users\EFRAIN LINARES\AppData\Local\Temp\ipykernel_18288\2352164090.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies["revenue"] = df_movies['revenue'].fillna(0)


,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Toy Story,7.7,5415.0
1,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Jumanji,6.9,2413.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Grumpier Old Men,6.5,92.0
3,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Waiting to Exhale,6.1,34.0
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Father of the Bride Part II,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",30840,en,"Yet another version of the classic epic, with ...",5.683753,"[{'name': 'Westdeutscher Rundfunk (WDR)', 'id'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1991-05-13,0.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Robin Hood,5.7,26.0
45462,NaN,0,"[{'id': 18, 'name': 'Drama'}]",111109,tl,An artist struggles to finish his work while a...,0.178241,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,Century of Birthing,9.0,3.0
45463,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Betrayal,3.8,6.0
45464,NaN,0,[],227506,en,"In a small town live two brothers, one a minis...",0.003503,"[{'name': 'Yermoliev', 'id': 88753}]","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",1917-10-21,0.0,87.0,[],Released,Satan Triumphant,0.0,0.0


In [15]:
# Cambiamos el tipo de dato a int o a float segun corresponda

df_movies['id'] = df_movies['id'].astype('int')
df_movies['budget']=df_movies['budget'].astype('float')
df_movies['vote_count']=df_movies['vote_count'].astype('int')
df_movies['revenue']=df_movies['revenue'].astype('int')
df_movies['budget']=df_movies['budget'].astype('int')

C:\Users\EFRAIN LINARES\AppData\Local\Temp\ipykernel_18288\630368958.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies['id'] = df_movies['id'].astype('int')
C:\Users\EFRAIN LINARES\AppData\Local\Temp\ipykernel_18288\630368958.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies['budget']=df_movies['budget'].astype('float')
C:\Users\EFRAIN LINARES\AppData\Local\Temp\ipykernel_18288\630368958.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

In [16]:
# Las fechas deben de tener un formato AAAA-MM-DD.
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], errors='coerce').dt.strftime('%Y-%m-%d')

# Se crea la columna "release_year" con los años de release_date

df_movies['release_year'] = pd.to_datetime(df_movies['release_date']).dt.year

C:\Users\EFRAIN LINARES\AppData\Local\Temp\ipykernel_18288\1761729555.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], errors='coerce').dt.strftime('%Y-%m-%d')
C:\Users\EFRAIN LINARES\AppData\Local\Temp\ipykernel_18288\1761729555.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies['release_year'] = pd.to_datetime(df_movies['release_date']).dt.year


In [17]:
#Se verifican los cambios ya hechos

df_movies.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count,release_year
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Toy Story,7.7,5415,1995
1,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Jumanji,6.9,2413,1995
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Grumpier Old Men,6.5,92,1995
3,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Waiting to Exhale,6.1,34,1995
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Father of the Bride Part II,5.7,173,1995


In [18]:
#Se crea una funcion para calcular el retorno de inversion.

def calculate_roi(row):
    try:
        revenue = (row['revenue'])
        budget = (row['budget'])
        return revenue / budget if budget != 0 else 0
    except (ValueError, TypeError):
        return 0


# Aplicar la funcion para que calcular el ROI y crear la nueva columna
df_movies['return'] = df_movies.apply(calculate_roi, axis=1)

C:\Users\EFRAIN LINARES\AppData\Local\Temp\ipykernel_18288\2552284735.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies['return'] = df_movies.apply(calculate_roi, axis=1)


<b>DESANIDADO</b>

In [19]:
#Para facilitar mi trabajo, he decidido desanidar las columnas, ya que creo que será más conveniente trabajar de esta manera. 
# Además, estoy creando una función que me permita expandir columnas en un DataFrame que contiene listas dentro de diccionarios.
def expand_columns(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else []) #Convierte el contenido de cada celda en la columna column de df de una cadena de texto que representa una lista de diccionarios a una lista de diccionarios Python, utilizando ast.literal_eval. Si el valor es nulo (pd.notnull(x)), se devuelve una lista vacía.
        # Expande la columna column en filas, de modo que cada elemento de la lista de diccionarios se convierte en una fila separada en df
        df = df.explode(column)
        # Normaliza la columna column después de expandirla, lo que significa que convierte cada diccionario dentro de la lista en una serie de columnas.
        col_df = pd.json_normalize(df[column])
        # Renombra las columnas normalizadas agregando el nombre original de la columna como prefijo, para evitar conflictos si hay nombres de columnas duplicados en diferentes listas de diccionarios.
        col_df = col_df.add_prefix(f'{column}_')
        # Elimina la columna original column del DataFrame df y la reemplaza con las columnas normalizadas (col_df), asegurándose de que el índice del DataFrame se reinicie para evitar problemas con el índice de las filas.
        df = df.drop(columns=[column]).reset_index(drop=True).join(col_df)
    return df

columns_to_expand = ['genres', 'production_companies', 'production_countries', 'spoken_languages','belongs_to_collection']

# Expandir las columnas de listas de diccionarios
df_movies = expand_columns(df_movies, columns_to_expand)

C:\Users\EFRAIN LINARES\AppData\Local\Temp\ipykernel_18288\3816972224.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else []) #Convierte el contenido de cada celda en la columna column de df de una cadena de texto que representa una lista de diccionarios a una lista de diccionarios Python, utilizando ast.literal_eval. Si el valor es nulo (pd.notnull(x)), se devuelve una lista vacía.


In [20]:
df_movies

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,title,...,release_year,return,genres_id,genres_name,production_companies_name,production_companies_id,production_countries_iso_3166_1,production_countries_name,spoken_languages_iso_639_1,spoken_languages_name
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81.0,Released,Toy Story,...,1995,12.451801,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English
1,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81.0,Released,Toy Story,...,1995,12.451801,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English
2,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81.0,Released,Toy Story,...,1995,12.451801,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English
3,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81.0,Released,Toy Story,...,1995,12.451801,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English
4,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81.0,Released,Toy Story,...,1995,12.451801,35.0,Comedy,Pixar Animation Studios,3.0,US,United States of America,en,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579083,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0,90.0,Released,Betrayal,...,2003,0.000000,28.0,Action,American World Pictures,6165.0,US,United States of America,en,English
579084,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0,90.0,Released,Betrayal,...,2003,0.000000,18.0,Drama,American World Pictures,6165.0,US,United States of America,en,English
579085,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0,90.0,Released,Betrayal,...,2003,0.000000,53.0,Thriller,American World Pictures,6165.0,US,United States of America,en,English
579086,0,227506,en,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0,87.0,Released,Satan Triumphant,...,1917,0.000000,NaN,NaN,Yermoliev,88753.0,RU,Russia,NaN,NaN


df_credits

In [27]:
#Se separan el dataframe de credits para hacer el desanidado mas eficiente
df_cast = df_credits[['id','cast']].copy()
df_crew = df_credits[['id','crew']].copy()

In [28]:
def expand_columns(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else []) #Convierte el contenido de cada celda en la columna column de df de una cadena de texto que representa una lista de diccionarios a una lista de diccionarios Python, utilizando ast.literal_eval. Si el valor es nulo (pd.notnull(x)), se devuelve una lista vacía.
        # Expande la columna column en filas, de modo que cada elemento de la lista de diccionarios se convierte en una fila separada en df
        df = df.explode(column)
        # Normaliza la columna column después de expandirla, lo que significa que convierte cada diccionario dentro de la lista en una serie de columnas.
        col_df = pd.json_normalize(df[column])
        # Renombra las columnas normalizadas agregando el nombre original de la columna como prefijo, para evitar conflictos si hay nombres de columnas duplicados en diferentes listas de diccionarios.
        col_df = col_df.add_prefix(f'{column}_')
        # Elimina la columna original column del DataFrame df y la reemplaza con las columnas normalizadas (col_df), asegurándose de que el índice del DataFrame se reinicie para evitar problemas con el índice de las filas.
        df = df.drop(columns=[column]).reset_index(drop=True).join(col_df)
    return df

columns_to_expand = ['cast']

df_credit_cast = expand_columns(df_cast, columns_to_expand)

In [29]:
def expand_columns(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else []) #Convierte el contenido de cada celda en la columna column de df de una cadena de texto que representa una lista de diccionarios a una lista de diccionarios Python, utilizando ast.literal_eval. Si el valor es nulo (pd.notnull(x)), se devuelve una lista vacía.
        # Expande la columna column en filas, de modo que cada elemento de la lista de diccionarios se convierte en una fila separada en df
        df = df.explode(column)
        # Normaliza la columna column después de expandirla, lo que significa que convierte cada diccionario dentro de la lista en una serie de columnas.
        col_df = pd.json_normalize(df[column])
        # Renombra las columnas normalizadas agregando el nombre original de la columna como prefijo, para evitar conflictos si hay nombres de columnas duplicados en diferentes listas de diccionarios.
        col_df = col_df.add_prefix(f'{column}_')
        # Elimina la columna original column del DataFrame df y la reemplaza con las columnas normalizadas (col_df), asegurándose de que el índice del DataFrame se reinicie para evitar problemas con el índice de las filas.
        df = df.drop(columns=[column]).reset_index(drop=True).join(col_df)
    return df

columns_to_expand = ['crew']

df_credit_crew = expand_columns(df_crew, columns_to_expand)

In [30]:
df_credit_cast

,id,cast_cast_id,cast_character,cast_credit_id,cast_gender,cast_id,cast_name,cast_order,cast_profile_path
0,862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,862,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,862,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,862,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,862,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg
...,...,...,...,...,...,...,...,...,...
564541,227506,3.0,,52fe4ea59251416c7515d7d9,1.0,1090923.0,Nathalie Lissenko,1.0,None
564542,227506,4.0,,52fe4ea59251416c7515d7dd,2.0,1136422.0,Pavel Pavlov,2.0,None
564543,227506,5.0,,52fe4ea59251416c7515d7e1,0.0,1261758.0,Aleksandr Chabrov,3.0,None
564544,227506,6.0,,52fe4ea59251416c7515d7e5,1.0,29199.0,Vera Orlova,4.0,/n1NXVGNzNxtqsMWxLT1h8GO8Kpi.jpg


In [31]:
df_credit_crew

,id,crew_credit_id,crew_department,crew_gender,crew_id,crew_job,crew_name,crew_profile_path
0,862,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,862,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,862,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,862,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,862,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg
...,...,...,...,...,...,...,...,...
464693,67758,52fe4776c3a368484e0c8399,Sound,0.0,549356.0,Original Music Composer,Richard McHugh,None
464694,67758,52fe4776c3a368484e0c839f,Camera,2.0,58818.0,Director of Photography,João Fernandes,None
464695,227506,533bccebc3a36844cf0011a7,Directing,0.0,1085341.0,Director,Yakov Protazanov,/yyjbGdCs2ZN6IlZNCfmBWyuRDlt.jpg
464696,227506,58ebbc26925141281908aa0a,Production,2.0,1195656.0,Producer,Joseph N. Ermolieff,None


Limpieza de los datos ya desanidados

In [33]:
#eliminar datos duplicados
df_movies = df_movies.drop_duplicates()
df_credit_cast = df_credit_cast.drop_duplicates()
df_credit_crew = df_credit_crew.drop_duplicates()

In [34]:
column_to_drop = ['genres_id', 'production_companies_id', 'production_countries_iso_3166_1', 'spoken_languages_iso_639_1']

df_movies = df_movies.drop(columns=column_to_drop)

df_movies.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,title,vote_average,vote_count,release_year,return,genres_name,production_companies_name,production_countries_name,spoken_languages_name
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81.0,Released,Toy Story,7.7,5415,1995,12.451801,Animation,Pixar Animation Studios,United States of America,English
4,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81.0,Released,Toy Story,7.7,5415,1995,12.451801,Comedy,Pixar Animation Studios,United States of America,English
8,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81.0,Released,Toy Story,7.7,5415,1995,12.451801,Family,Pixar Animation Studios,United States of America,English
12,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249,104.0,Released,Jumanji,6.9,2413,1995,4.043035,Adventure,TriStar Pictures,United States of America,English
13,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249,104.0,Released,Jumanji,6.9,2413,1995,4.043035,Adventure,TriStar Pictures,United States of America,Français


In [35]:
column_to_drop = ['cast_cast_id', 'cast_credit_id', 'cast_profile_path', 'cast_gender','cast_id']

df_credit_cast = df_credit_cast.drop(columns=column_to_drop)
df_credit_cast.head()

,id,cast_character,cast_name,cast_order
0,862,Woody (voice),Tom Hanks,0.0
1,862,Buzz Lightyear (voice),Tim Allen,1.0
2,862,Mr. Potato Head (voice),Don Rickles,2.0
3,862,Slinky Dog (voice),Jim Varney,3.0
4,862,Rex (voice),Wallace Shawn,4.0


In [36]:
column_to_drop = ['crew_credit_id', 'crew_gender', 'crew_department', 'crew_id', 'crew_profile_path']

df_credit_crew = df_credit_crew.drop(columns=column_to_drop)

In [37]:
#Estoy eliminando los actores que no considero como principales.
df_credit_cast = df_credit_cast[df_credit_cast['cast_order'] <= 3]

In [38]:
#En la columna "crew_job", vamos a conservar únicamente los registros donde el cargo sea "Director".
df_credit_crew = df_credit_crew[df_credit_crew['crew_job'] == 'Director']

In [39]:
#Elimino la columa crew_job que ya no aporta más información.
df_credit_crew = df_credit_crew.drop(columns='crew_job')

In [40]:
df_movies.dropna(subset=['genres_name'], inplace=True)
df_movies.dropna(subset=['production_countries_name'], inplace=True)
df_movies.dropna(subset=['production_companies_name'], inplace=True)
df_movies.dropna(subset=['spoken_languages_name'], inplace=True)

In [41]:
#Eliminamos las películas que sean más viejas que 1970
df_movies = df_movies[df_movies['release_year'] >= 1970]

In [42]:
#Elimino las filas duplicadas teniendo en cuenta únicamente las columnas "title", "genres_name" y "production_companies_name". 
# Creo que estas columnas serán útiles para el aprendizaje automático, mientras que otras como el lenguaje no son tan relevantes en este contexto.
df_movies = df_movies.drop_duplicates(subset=['title', 'genres_name', 'production_companies_name'])

In [43]:
df_movies['budget'] = pd.to_numeric(df_movies['budget'], errors='coerce')
df_movies['popularity'] = pd.to_numeric(df_movies['popularity'], errors='coerce')
df_movies['revenue'] = pd.to_numeric(df_movies['revenue'], errors='coerce')
df_movies['runtime'] = pd.to_numeric(df_movies['runtime'], errors='coerce')
df_movies['vote_average'] = pd.to_numeric(df_movies['vote_average'], errors='coerce')
df_movies['vote_count'] = pd.to_numeric(df_movies['vote_count'], errors='coerce')
df_movies['release_year'] = pd.to_numeric(df_movies['release_year'], errors='coerce')
df_movies['return'] = pd.to_numeric(df_movies['return'], errors='coerce')

df_credit_cast['cast_order'] = pd.to_numeric(df_credit_cast['cast_order'], errors='coerce')

# Guardar los DataFrames en formato Parquet
df_movies.to_parquet('movies_clean.parquet', index=False)
df_credit_cast.to_parquet('credit_cast_clean.parquet', index=False)
df_credit_crew.to_parquet('credit_crew_clean.parquet', index=False)